In [1]:
from transformers import AutoTokenizer, DistilBertModel
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
model = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased")
model.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [2]:
from utils.DevConf import DevConf
devConf = DevConf('mps')

In [3]:
from utils.AttnBlocksConf import AttnBlocksConf


In [4]:
from model.BertDecoder.SentiClassifier import SentiClassifier
from model.CombinationModel import CombinationModel
from torch import nn

In [5]:
from utils.const import BlockType

In [6]:
mapper = SentiClassifier(AttnBlocksConf(768, 12, 6), BlockType.PARALLEL,devConf=devConf)
cModel = CombinationModel(tokenizer=tokenizer, distilBert=model, decoder=mapper, outputProject=nn.Linear(768, 5, device=devConf.device))

In [7]:
ans = cModel.forward("嗨", True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
print(ans)

tensor([[0.2688, 0.1238, 0.1436, 0.3052, 0.1586]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
